In [1]:
from gurobipy import GRB
import gurobipy as gb

In [2]:
# Create the optimization model
model = gb.Model("Transshipment Problem")

Restricted license - for non-production use only - expires 2025-11-24


In [3]:
# A list of list of costs
source_costs = [[11, 10, 26, 29], [9, 12, 27, 26]]
trans_costs = [[12, 16], [13, 15]]

In [4]:
# Create the a single class of decision variables where
# From = {𝑫𝒂,𝑯𝒐} and To = {𝑪𝒉,𝑳𝑨,𝑺𝑭,𝑵𝒀}.
x = model.addVars(2, 4, lb=0, vtype=GRB.CONTINUOUS, name="Source Nodes")
# From = {𝑪𝒉,𝑳𝑨} and To = {𝑺𝑭,𝑵𝒀}.
y = model.addVars(2, 2, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment Nodes")

In [5]:
# The objective function
source_objective = gb.quicksum(source_costs[i][j]*x[i,j] for i in range(2) for j in range(4))
trans_objective = gb.quicksum(trans_costs[i][j]*y[i,j] for i in range(2) for j in range(2))
model.setObjective(source_objective + trans_objective, GRB.MINIMIZE)

In [6]:
# Add the supply constraints from source nodes
model.addConstr(gb.quicksum(x[0,j] for j in range(4)) <= 200, name="Supply Constraint 1")
model.addConstr(gb.quicksum(x[1,j] for j in range(4)) <= 160, name="Supply Constraint 2")

<gurobi.Constr *Awaiting Model Update*>

In [7]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) <= 90, name="Transship Capacity 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) <= 80, name="Transship Capacity 2")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(x[i,0] for i in range(2)) == gb.quicksum(y[0,k] for k in range(2)), name="Flow Balance 1")
model.addConstr(gb.quicksum(x[i,1] for i in range(2)) == gb.quicksum(y[1,k] for k in range(2)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [9]:
# Add the demand constraints
model.addConstr(gb.quicksum(x[i,2] + y[i,0] for i in range(2)) == 140, name="Demand Constraint 1")
model.addConstr(gb.quicksum(x[i,3] + y[i,1] for i in range(2)) == 140, name="Demand Constraint 2")

<gurobi.Constr *Awaiting Model Update*>

In [10]:
# Ratio constraint
model.addConstr(0.6*gb.quicksum(y[i,j] for i in range(2) for j in range(2)) <= 0.4*gb.quicksum(x[i,j] for i in range(2) for j in range(2,4)), name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [11]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 9 rows, 12 columns and 36 nonzeros
Model fingerprint: 0x828d4180
Coefficient statistics:
  Matrix range     [4e-01, 1e+00]
  Objective range  [9e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 2e+02]
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7800000e+03   4.480000e+02   0.000000e+00      0s
       5    6.9040000e+03   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.904000000e+03


In [12]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Number of Decision Variables:  12


In [13]:
# Value of the objective function
print("Total Transportation cost: ", model.objVal)

Total Transportation cost:  6904.0


In [14]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Source Nodes[0,1]           46 
Source Nodes[0,2]           74 
Source Nodes[1,0]           66 
Source Nodes[1,3]           94 
Transshipment Nodes[0,0]           66 
Transshipment Nodes[1,1]           46 
None
